# Setup

In [10]:
import json
from functools import reduce

from openai import OpenAI
from outlines.fsm.json_schema import build_regex_from_schema, get_schema_from_signature

In [3]:
client = OpenAI(base_url="http://localhost:8000/v1", api_key="-")
client

# Tool To Regex

In [11]:
def regex_or(pattern1, pattern2):
    return f"(?:{pattern1}|{pattern2})"

In [12]:
def sometime_guide(function_regex, start_word="<tool_call>", end_word="</tool_call>"):
    """
    Only do guided generation sometimes, in between start_word and end_word.
    """
    return f".*?(?={start_word}){start_word}({function_regex}).*?(?={end_word}){end_word}.*"

In [ ]:
def tool_to_regex(tool, whitespace_pattern=None, test_category=None):

    if isinstance(tool, list):
        regex_strs = [tool_to_regex(_tool, whitespace_pattern=whitespace_pattern, test_category=test_category) for _tool in tool]
        regex_str = reduce(regex_or, regex_strs)
    elif isinstance(tool, dict) and list(tool.keys()) == ['name', 'description', 'parameters']:
        schema_str = bfcl_function_to_schema(tool, test_category)
    elif callable(tool):
        schema_str = json.dumps(get_schema_from_signature(tool))


# Prompt

In [4]:
messages = [
    {"role": "user", "content": "Classify this sentiment: vLLM is wonderful!"}
  ]

In [5]:
completion = client.chat.completions.create(
  model="databricks/dbrx-instruct",
  messages=messages,
  #extra_body=dict(guided_regex=TEST_REGEX, guided_decoding_backend="outlines"),
  )

raw_text = completion.choices[0].message.content